# Lab 3: Word Embeddings and Language Modelling

Adam Ek

In this lab we'll explore constructing *static* word embeddings (i.e. word2vec) and building language models. We'll also evaluate these systems on intermediate tasks, namely word similarity and identifying "good" and "bad" sentences.

* For this we'll use pytorch. Some basic operations that will be useful can be found here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations
* In general: we are not interested in getting state-of-the-art performance :) focus on the implementation and not results of your model. For this reason, you can use a subset of the dataset: the first 5000-10 000 sentences or so, on linux/mac: ```head -n 10000 inputfile > outputfile```. 
* If possible, use the MLTGpu, it will make everything faster :)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim
from torchtext.legacy.data import Field, BucketIterator, Iterator, TabularDataset

#from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
from IPython import embed

import numpy as np


# for gpu, replace "cpu" with "cuda:n" where n is the index of the GPU
device = torch.device('cpu')

# Word2Vec embeddings

In this first part we'll construct a word2vec model which will give us *static* word embeddings (that is, they are fixed after training).

After we've trained our model we will evaluate the embeddings obtained on a word similarity task.

## Formatting data


First we need to load some data, you can download the file on canvas under files/03-lab-data/wiki-corpus.txt. The file contains 50 000 sentences randomly selected from the complete wikipedia. Each line in the file contains one sentence. The sentences are whitespace tokenized.

Your first task is to create a dataset suitable for word2vec. That is, we define some ```window_size``` then iterate over all sentences in the dataset, putting the center word in one field and the context words in another (separate the fields with ```tab```).

For example, the sentece "this is a lab" with ```window size = 4``` will be formatted as:
```
center, context
---------------------
this    is a lab
is      this a lab
a       this is lab
lab     this is a
```

this will be our training examples when training the word2vec model.

[3 marks]

In [16]:
import string
import pandas as pd

s = "string. With. Punctuation?"
table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}
new_s = s.translate(table) 

data_path = 'wiki-corpus.txt'
WINDOW_SIZE = 4
def corpus_reader(data_path):
    with open(data_path, encoding="utf-8") as f:
        lines = f.readlines()
        centerWord_contextWords = []
        exceptions_short_sentences_count = 0
        for lineCount,line in enumerate(lines):
            #print (line)
            table = str.maketrans(dict.fromkeys(string.punctuation)) #start punctuation removal
            #print ("table:",table)
            line = line.translate(table) #remove punctuation
            line = line.split()
            #line = ["this","is","a","lab","with","several","words","altogether","making","a","sentence","that","is","long"]
            #line = ["this","is","a","lab"]
            #print ("line:",line)
            #print ("len of line:",len(line))
            for i,word in enumerate(line):
                context_words=[]
                #print ("center word at:",i,"(",word,")")
                if i==0:
                    start_place = 0
                    end_place = WINDOW_SIZE
                    for ii in range(start_place,end_place):
                        #print ("For word",i,"take context word at:",ii)
                        try:
                            context_words.append(line[ii])
                        except IndexError:
                            #print ("exception occured.\nline is:",line,"\nii is: -----",ii)
                            exceptions_short_sentences_count += 1
                else:
                    start_place = i-int(WINDOW_SIZE/2)
                    end_place = i+int(WINDOW_SIZE/2)
                    #print ("start place:", start_place)
                    #print ("end place:",end_place)
                    j=start_place
                    while j<end_place:
                        #print ("j:",j)
                        if start_place<0:
                            start_place += 1
                            end_place += 1
                            #print ("start place now:",start_place,", end place now:",end_place)
                            j += 1
                            continue
                        elif end_place > len(line):
                            #print ("word at i:",i,", end_place:",end_place)
                            for k in range(start_place-1,end_place-1):
                                #print ("k position:",k)
                                #print ("and lastly for word: (",word,"):",line[k])
                                try:
                                    context_words.append(line[k])
                                except IndexError:
                                    pass
                            break
                        #print ("For word",i,"(",word,")","take context word at:",j,"(",line[j],")")
                        context_words.append(line[j])
                        j += 1
                context_words.remove(word)
                #print ("center word:",word,", context words:",context_words)
                centerWord_contextWords.append([word," ".join(context_words)])
            #if lineCount==1:
            #    print ("line 2 centerWord_contextWords:",centerWord_contextWords)
        print ("- note: number of shorter senteces -- than window size -- encountered:",exceptions_short_sentences_count)
        print ("- done creating center word and its context words for window size:",WINDOW_SIZE,"for all lines (",len(lines),") lines")
        df = pd.DataFrame(centerWord_contextWords, columns=['center','context'])
        df.to_csv(data_path+".csv",index=False, sep='\t')
        print ("- done saving center words/context words into csv file. ")
corpus_reader(data_path)

- note: number of shorter senteces -- than window size -- encountered: 918
- done creating center word and its context words for window size: 4 for all lines ( 50000 ) lines
- done saving center words/context words into csv file. 


In [17]:
pd.read_csv("wiki-corpus.txt.csv",sep='\t')

,center,context
0,Anarchist,historian George Woodcock
1,historian,Anarchist George Woodcock
2,George,Anarchist historian Woodcock
3,Woodcock,historian George reports
4,reports,George Woodcock that
5,that,Woodcock reports The
6,The,reports that annual
7,annual,that The Congress
8,Congress,The annual of
9,of,annual Congress the


We sampled 50 000 senteces completely random from the *whole* wikipedia for our training data. Give some reasons why this is good, and why it might be bad. (*note*: We'll have a few questions like these, one or two reasons for and against is sufficient)

[2 marks]

In [ ]:
Sampling 50000 diverse Wikipedia sentences is good because it covers a large variation of sentence types with different words and lengths.

Hoever it may not be ideal as it would be too general and would not be enough for such general task; since a general task requires a very large corpus with several times the magnitude of what is currently being sampled.

### Loading the data

We now need to load the data in an appropriate format for torchtext (https://torchtext.readthedocs.io/en/latest/). We'll use PyText for this and it'll follow the same structure as I showed you in the lecture (remember to lower-case all tokens). Create a function which returns a (bucket)iterator of the training data, and the vocabulary object (```Field```). 

(*hint1*: you can format the data such that the center word always is first, then you only need to use one field)

(*hint2*: the code I showed you during the leture is available in /files/pytorch_tutorial/ on canvas)

[4 marks]

In [79]:
def get_data(dataFilePath):
    whitespacer = lambda x: x.split(' ')

    # "fields" that process the different columns in our CSV files
    TOKENS = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) # enforce the (batch, words) structure

    # read the csv files
    train = TabularDataset(path = dataFilePath,
                           format = 'csv',
                           fields = [('center', TOKENS),
                                     ('context', TOKENS)],
                           skip_header       = True,
                           csv_reader_params = {'delimiter':'\t',
                                                'quotechar':'⅞'})
    
    # build vocabularies based on what our csv files contained and create word2id mapping
    TOKENS.build_vocab(train, min_freq=1)


    # create batches from our data, and shuffle them for each epoch
    train_iter = BucketIterator(dataset=train,
                                batch_size        = 8,
                                #sort_key=lambda x: len(x.comment_text), #https://github.com/pytorch/text/issues/474
                                sort_within_batch = False, #changed from True to False
                                shuffle           = True,
                                device            = device)

    return train_iter, TOKENS.vocab
    

In [80]:
dataset, vocab = get_data("wiki-corpus.txt.csv")

We lower-cased all tokens above; give some reasons why this is a good idea, and why it may be harmful to our embeddings.

[2 marks]

- By lower-casing the tokens we are summing up all instances of each token into one unit. For example if we have three instances of "John" & "john" & "johN", we can sum all of them into one "john" with frequency of 3, thus improving our learning and prediction.
- However it might be harmful if two different meanings are meant by the lower-case and the capitalized version of one word, such as names of companies that may have a dictionary meaning as well and cannot be ditinguished as company names if they are lower-cased.

## Word Embeddings Model

We will implement the CBOW model for constructing word embedding models.

In [74]:
import torch.optim as optim

In the CBOW model we try to predict the center word based on the context. That is, we take as input ```n``` context words, encode them as vectors, then combine them by summation. This will give us one embedding. We then use this embedding to predict *which* word in our vocabuary is the most likely center word. 

Implement this model 

[7 marks]

In [81]:
class CBOWModel(nn.Module):
    def __init__(self, num_words, num_dim): #...
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(num_words, num_dim) #...
        self.prediction = nn.Linear(num_dim, num_words) #...
    
    def forward(self, context):
        embedded_context = self.embeddings(context) #...
        embedded_context = self.projection_function(embedded_context)
        output = self.linear1(embedded_context)
        log_probs = F.log_softmax(output, dim=1)
        return log_probs
        
    def projection_function(self, xs):
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        xs_sum = torch.sum(xs, dim=1)
        return xs_sum

In [62]:
CBOWModel(len(vocab),50).to(device)

CBOWModel(
  (embeddings): Embedding(77632, 50)
  (prediction): Linear(in_features=50, out_features=77632, bias=True)
)

In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, ...):
        super(CBOWModel, self).__init__()
        self.embeddings = ...
        self.prediction = ...
    
    def forward(self, context):
        embedded_context = ...
        projection = ...
        predictions = ...
        
        return predictions
        
    def projection_function(self, xs):
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        xs_sum = ...
        return xs_sum

Now we need to train the models. First we define which hyperparameters to use. (You can change these, for example when *developing* your model you can use a batch size of 2 and a very low dimensionality (say 10), just to speed things up). When actually training your model *fo real*, you can use a batch size of [8,16,32,64], and embedding dimensionality of [128,256].

In [82]:
# you can change these numbers to suit your needs :)
word_embeddings_hyperparameters = {'epochs':3,
                                   'batch_size':16,
                                   'embedding_size':128,
                                   'learning_rate':0.001,
                                   'embedding_dim':128}

Train your model. Iterate over the dataset, get outputs from your model, calculate loss and backpropagate.

We mentioned in the lecture that we use Negative Log Likelihood (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) loss to train Word2Vec model. In this lab we'll take a shortcut when *training* and use Cross Entropy Loss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), basically it combines ```log_softmax``` and ```NLLLoss```. So what your model should output is a *score* for each word in our vocabulary. The ```CrossEntropyLoss``` will then assign probabilities and calculate the negative log likelihood loss.

[3 marks]

In [83]:
# load data
dataset, vocab = get_data("wiki-corpus.txt.csv")

# build model and construct loss/optimizer
cbow_model = CBOWModel(len(vocab), word_embeddings_hyperparameters['embedding_dim'])
cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])

# start training loop
total_loss = 0
for epoch in range(word_embeddings_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        context = batch.context
        target_word = batch.center
        print ("context:",context)
        print ("target_word:",target_word)
        
        # send your batch of sentences to the model
        #####output = cbow_model(context)
        
        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        ####loss = loss_fn(output,)
        ####total_loss += loss.item()
        
        # print average loss for the epoch
        ####print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        ####...
        
        # update parameters
        ####...
        
        # reset gradients
        ####...
    print()
        

        [   93,  8688,     2],
        [ 4938,    25, 58440],
        [ 5764, 16826,  1039],
        [    3,  6969,    16],
        [  905,    50,     7],
        [    2,  3318,   751]])
target_word: tensor([[    3],
        [  213],
        [58292],
        [  849],
        [   12],
        [    5],
        [   66],
        [  147]])
context: tensor([[ 1284,     8,    16],
        [  468, 10079, 13844],
        [ 5559,    23,  1228],
        [   25, 23350,   231],
        [   12,  1162,  6426],
        [    4,   514,    60],
        [    5,  3959, 42350],
        [    7,  4247,  2285]])
target_word: tensor([[ 1801],
        [48064],
        [    4],
        [   27],
        [   73],
        [ 5858],
        [23438],
        [ 1040]])
context: tensor([[    4, 16419,   313],
        [ 1161,     3,   186],
        [   17,   197,   342],
        [    9,  4011,     4],
        [    2,    90,    11],
        [   84,  7868,     3],
        [   67,     2,  1476],
        [   14,  1889,     4]

KeyboardInterrupt: 

## Evaluating the model

We will evaluate the model on a dataset of word similarities, WordSim353 (http://alfonseca.org/eng/research/wordsim353.html , also avalable in vanvas under files/03-l). The first thing we need to do is read the dataset and translate it to integers. What we'll do is to reuse the ```Field``` that records word indexes (the second output of ```get_data()```) and use it to parse the file.

The wordsim data is structured as follows:

```
word1 word2 score
...
```


The ```Field``` we got from ```read_data()``` has two built-in functions, ```stoi``` which maps a string to an integer and ```itos``` which maps an integer to a string. 

What our datareader needs to do is: 

```
for line in file:
    word1, word2, score = file.split()
    # encode word1 and word2 as integers
    word1_idx = vocab.vocab.stoi[word1]
    word2_idx = vocab.vocab.stoi[word2]
```

when we have the integers for ```word_1``` and ```word2``` we'll compute the similarity between their word embeddings with *cosine simlarity*. We can obtain the embeddings by querying the embedding layer of the model.

We calculate the cosine similarity for each word pair in the dataset, then compute the pearson correlation between the similarities we obtained with the scores given in the dataset. 

[4 marks]

In [ ]:
# your code goes here

def read_wordsim(path, vocab, embeddings):
    dataset_sims = []
    model_sims = []
    with open(path) as f:
        for line in f:
            word1, word2, score = f.split()
            
            score = float(score)
            dataset_sims.append(score)
            
            # get the index for the word
            word1_idx = ...
            word2_idx = ...
            
            # get the embedding of the word
            word1_emb = ...
            word2_emb = ...
            
            # compute cosine similarity, we'll use the version included in pytorch functional
            # https://pytorch.org/docs/master/generated/torch.nn.functional.cosine_similarity.html
            cosine_similarity = F.cosine_similarity(...)
            
            model_sims.append(cosine_similarity.item())
    
    return dataset_sims, model_sims

path = 'wordsim_similarity_goldstandard.txt'
data, model = read_wordsim(...)
pearson_correlation = np.corrcoef(data, model)
            
# the non-diagonals give the pearson correlation,
print(pearson_correlation)

Do you think the model performs good or bad? Why?

[3 marks]

Select the 10 best and 10 worst performing word pairs, can you see any patterns that explain why *these* are the best and worst word pairs?

[3 marks]

Suggest some ways of improving the model we apply to WordSim353.

[3 marks]

If we consider a scenario where we use these embeddings in a downstream task, for example sentiment analysis (roughly: determining whether a sentence is positive or negative). 

Give some examples why the sentiment analysis model would benefit from our embeddnings and one examples why our embeddings could hur the performance of the sentiment model.

[3 marks]

# Language modeling

In this second part we'll build a simple LSTM language model. Your task is to construct a model which takes a sentence as input and predict the next word for each word in the sentence. For this you'll use the ```LSTM``` class provided by PyTorch (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). You can read more about the LSTM here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

NOTE!!!: Use the same dataset (wiki-cropus.txt) as before.

Our setup is similar to before, we first encode the words as distributed representations then pass these to the LSTM and for each output we predict the next word.

For this we'll build a new dataloader with torchtext, the file we pass to the dataloader should contain one sentence per line, with words separated by whitespace.

```
word_1, ..., word_n
word_1, ..., word_k
...
```

in this dataloader you want to make sure that each sentence begins with a ```<start>``` token and ends with a ```<end>``` token, there is a keyword argument in ```Field``` for this :). But other than that, as before you read the dataset and output a iterator over the dataset and a vocabulary. 

Implement the dataloader, language model and the training loop (the training loop will basically be the same as for word2vec).

[12 marks]

In [ ]:
# you can change these numbers to suit your needs as before :)
lm_hyperparameters = {'epochs':3,
                      'batch_size':16,
                      'learning_rate':0.001,
                      'embedding_dim':128,
                      'output_dim':128}

In [ ]:
data_path = 'wiki-corpus.txt'
def get_data():
    # your code here, roughly the same as for the word2vec dataloader

In [ ]:
class LM_withLSTM(nn.Module):
    def __init__(...):
        super(LM_withLSTM, self).__init__()
        self.embeddings = ...
        self.LSTM = ...
        self.predict_word = ...
    
    def forward(self, seq):
        embedded_seq = ...
        timestep_reprentation, *_ = ...
        predicted_words = ...
        
        return predicted_words

In [ ]:
# load data
dataset, vocab = get_data(...)

# build model and construct loss/optimizer
lm_model = LM_withLSTM(len(vocab), 
                       lm_hyperparameters['embedding_dim'],
                       lm_hyperparameters['output_dim'])
lm_model.to(device)

loss_fn = CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=lm_hyperparameters['lr'])

# start training loop
total_loss = 0
for epoch in range(lm_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        # the strucure for each BATCH is:
        # <start>, w0, ..., wn, <end>
        sentence = batch.sentence
        
        # when training the model, at each input we predict the *NEXT* token
        # consequently there is nothing to predict when we give the model 
        # <end> as input. 
        # thus, we do not want to give <end> as input to the model, select 
        # from each batch all tokens except the last. 
        # tip: use pytorch indexing/slicing (same as numpy) 
        # (https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html#operations-on-tensors)
        # (https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)
        input_sentence = ...
        
        # send your batch of sentences to the model
        output = lm_model(input_sentence)
        
        # for each output, the model predict the NEXT token, so we have to reshape 
        # our dataset again. On timestep t, we evaluate on token t+1. That is,
        # we never predict the <start> token ;) so this time, we select all but the first 
        # token from sentences (that is, all the tokens that we predict)
        gold_data = ...
        
        # the shape of the output and sentence variable need to be changed,
        # for the loss function. Details are in the documentation.
        # You can use .view(...,...) to reshape the tensors  
        loss = loss_fn(...)
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        ...
        
        # update parameters
        ...
        
        # reset gradients
        ...
    print()

### Evaluating the language model

We'll evaluate our model using the BLiMP dataset (https://github.com/alexwarstadt/blimp). The BLiMP dataset contains sets of linguistic minimal pairs for various syntactic and semantic phenomena, We'll evaluate our model on *existential quantifiers* (link: https://github.com/alexwarstadt/blimp/blob/master/data/existential_there_quantifiers_1.jsonl). This data, as the name suggests, investigate whether language models assign higher probability to *correct* usage of there-quantifiers. 

An example entry in the dataset is: 

```
{"sentence_good": "There was a documentary about music irritating Allison.", "sentence_bad": "There was each documentary about music irritating Allison.", "field": "semantics", "linguistics_term": "quantifiers", "UID": "existential_there_quantifiers_1", "simple_LM_method": true, "one_prefix_method": false, "two_prefix_method": false, "lexically_identical": false, "pairID": "0"}
```

Download the dataset and build a datareader (similar to what you did for word2vec). The dataset structure you should aim for is (you don't need to worry about the other keys for this assignment):

```
good_sentence_1, bad_sentence_1
...
```

your task now is to compare the probability assigned to the good sentence with to the probability assigned to the bad sentence. To compute a probability for a sentence we consider the product of the probabilities assigned to the *gold* tokens, remember, at timestep ```t``` we're predicting which token comes *next* e.g. ```t+1``` (basically, you do the same thing as you did when training).

In rough pseudo code what your code should do is:

```
accuracy = []
for good_sentence, bad_sentence in dataset:
    gs_lm_output = LanguageModel(good_sentence)
    gs_token_probabilities = softmax(gs_lm_output)
    gs_sentence_probability = product(gs_token_probabilities[GOLD_TOKENS])

    bs_lm_output = LanguageModel(bad_sentence)
    bs_token_probabilities = softmax(bs_lm_output)
    bs_sentence_probability = product(bs_token_probabilities[GOLD_TOKENS])

    # int(True) = 1 and int(False) = 0
    is_correct = int(gs_sentence_probability > bs_sentence_probability)
    accuracy.append(is_correct)

print(numpy.mean(accuracy))
    
```

[6 marks]

In [ ]:
# your code goes here
import json

def evaluate_model(path, vocab, model):
    
    accuracy = []
    with open(path) as f:
        # iterate over one pair of sentences at a time
        for line in f:
            # load the data
            data = json.loads(line)
            good_s = data['sentence_good']
            bad_s = data['sentence_bad']
            
            # the data is tokenized as whitespace
            tok_good_s = ...
            tok_bad_s = ...
            
            # encode your words as integers using the vocab from the dataloader, size is (S)
            # we use unsqueeze to create the batch dimension 
            # in this case our input is only ONE batch, so the size of the tensor becomes: 
            # (S) -> (1, S) as the model expects batches
            enc_good_s = torch.tensor([_ for x in tok_good_s], device=device).unsqueeze(0)
            enc_bad_s = torch.tensor([_ for x in tok_bad_s], device=device).unsqueeze(0)
            
            # pass your encoded sentences to the model and predict the next tokens
            good_s = LM_withLSTM(enc_good_s)
            bad_s = LM_withLSTM(enc_bad_s)
            
            # get probabilities with softmax
            gs_probs = F.softmax(...)
            bs_probs = F.softmax(...)
            
            # select the probability of the gold tokens
            gs_sent_prob = find_token_probs(gs_probs, enc_good_s)
            bs_sent_prob = find_token_probs(bs_probs, enc_bad_s)
            
            accuracy.append(int(gs_sent_prob>bs_sent_prob))
            
    return accuracy
            
def find_token_probs(model_probs, encoded_sentece):
    probs = []

    # iterate over the tokens in your encoded sentence
    for token, gold_token in enumerate(encoded_sentece):
        # select the probability of the gold tokens and save
        # hint: pytorch indexing is helpful here ;)
        prob = ...
        probs.append(prob)
    sentence_prob = ...
    return sentence_prob

path = 'existential_there_quantifiers_1.jsonl'
accuracy = evaluate_model(path, ..., ...)

print('Final accuracy:')
print(np.round(np.mean(accuracy), 3))


### Analysis

Our model get some score, say, 55% correct predictions. Is this good? Suggest some *baseline* (i.e. a stupid "model" we hope ours is better than) we can compare the model against.

[3 marks]

Suggest some improvements you could make to your language model.

[3 marks]

Suggest some other metrics we can use to evaluate our system

[2 marks]

# Literature


Neural architectures:
* Y. Bengio, R. Ducharme, P. Vincent, and C. Janvin. A neural probabilistic language model. (Links to an external site.) Journal of Machine Learning Research, 3(6):1137–1155, 2003. (Sections 3 and 4 are less relevant today and hence you can glance through them quickly. Instead, look at the Mikolov papers where they describe training word embeddings with the current neural network architectures.)
* T. Mikolov, K. Chen, G. Corrado, and J. Dean. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013.
* T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.
    


Total marks: 63